In [ ]:
import healpy as hp
import numpy as np

In [ ]:
alm = hp.read_alm("/simons/scratch/zonca/simonsobs/so_pysm_models_data/equatorial/websky/0.3/unlensed_alm_seed1.fits", (1,2,3))

In [ ]:
alm

In [ ]:
m = hp.alm2map(alm.astype(np.complex), nside=4096)

In [ ]:
hp.fit_dipole(m[0])

In [ ]:
m[0] = hp.remove_dipole(m[0])

In [ ]:
hp.fit_dipole(m[0])

In [ ]:
import dipole

https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/Map-making#HFI_2018_Solar_dipole

HFI solar dipole 2018

v = (369.8150 ± 0.0010) km s-1

A = (3362.08 ± 0.09 (stat.) ± 0.45 (syst.) ± 0.32 (cal.) μK

l = (264.021 ± 0.003 (stat.) ± 0.0079 (syst.) )°

b = (48.253 ± 0.001 (stat.) ± 0.0037 (syst.) )°



In [ ]:
dip_lon = 264.021
dip_lat = 48.253

In [ ]:
import pysm.units as u
from astropy.constants import c

In [ ]:
c

In [ ]:
T_CMB = 2.72548  * u.K
vel = 369815 * (u.m/u.s)
beta = vel / c
gamma=1/np.sqrt(1-beta**2)
cosdir = 1
T_dipole_CMB = T_CMB / (gamma * ( 1 - beta * cosdir ))
solar_dipole_amplitude = T_dipole_CMB - T_CMB

In [ ]:
solar_dipole_amplitude.to(u.uK)

In [ ]:
gal2eq = hp.Rotator(coord="GC")

In [ ]:
solar_dipole_vec_gal = gal2eq(hp.ang2vec(dip_lon, dip_lat, lonlat=True))

In [ ]:
npix = len(m[0])
nside = hp.npix2nside(npix)


In [ ]:
vec = np.array(hp.pix2vec(nside, np.arange(npix)))

In [ ]:
vec.shape

In [ ]:
cosdir = np.dot(solar_dipole_vec_gal, vec)

In [ ]:
solar_dipole_map = T_CMB / (gamma * ( 1 - beta * cosdir )) - T_CMB

In [ ]:
cosdir.max()

In [ ]:
%matplotlib inline

In [ ]:
solar_dipole_map.max()

In [ ]:
hp.mollview(solar_dipole_map)

In [ ]:
hp.mollview(solar_dipole_map, coord="GC")

In [ ]:
hp.fit_dipole(solar_dipole_map)

In [ ]:
m.max()

In [ ]:
m[0] += solar_dipole_map.value * 1e6

In [ ]:
m[0] = hp.remove_monopole(m[0])

In [ ]:
for i,pol in enumerate("IQU"):
    hp.write_map(f"/simons/scratch/zonca/simonsobs/so_pysm_models_data/equatorial/websky/0.3/map_{pol}_unlensed_alm_seed1_solardipole_nside{nside}.fits", 
                 m[i], overwrite=True)

In [ ]:
ls /global/homes/z/zonca/prjcmb/www/so_pysm_models_data/websky/0.3/

In [ ]:
hp.mollview(m[0])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cl = hp.anafast(m[0])

In [ ]:
plt.loglog(cl)
plt.loglog(hp.alm2cl(alm[0].astype(np.complex)))